# Algorithm Performance

This notebook assesses the performance of the tdr algorithm. The metrics used are **Mean Squared Error (MSE), Root Mean Squared Error (RMSE), Mean Absolute Error (MAE) and Inner Ratio Metric (IRM)**.

In [1]:
import os
import numpy as np
import configparser
import pandas as pd
import tdr_utils as ut

In [2]:
# Get parameters from configuration file
config = configparser.ConfigParser()
config.read('tdr-parameters.ini')

threshold = int(config['signal']['threshold'])
frequency = float(config['signal']['frequency'])
duty_cycle = float(config['signal']['duty_cycle'])
num_samples = int(config['signal']['num_samples'])
sampling_rate = float(config['signal']['sampling_rate'])

num_edges = int(config['search']['num_edges']) #number of rising and falling, each, edges to search for
overlap_offset = int(config['search']['overlap_offset']) #number of samples to offset overlap threshold with

win_size = int(config['win_interest']['win_size']) 
prior_samples = int(config['win_interest']['prior_samples']) # samples to consider before rising/falling edge index


overlap_threshold = duty_cycle * (sampling_rate / frequency) - overlap_offset
t = np.linspace(0, num_samples / sampling_rate, num=num_samples) * 1000 #time in milliseconds

In [3]:
e_values = [] #expected distance values
p_values = [] #predicted distance values
excel_file = 'files-paths.xlsx'
xls = pd.ExcelFile(excel_file)
for name in xls.sheet_names:
    paths = pd.read_excel(excel_file, sheet_name=name)['paths'].tolist()
    
    for path in paths:
        v = ut.file_loader(path) #load the file
        valid_file = ut.file_validity(v, num_samples, threshold) #check if there was a bus conflict while saving the file

        if valid_file:
            edges_indices = ut.edges_detection(v, t, num_edges, win_size, prior_samples, overlap_threshold)
            avg_t_delay,  t_delays = ut.avg_delay(edges_indices, win_size, v, t, prior_samples, display=False)
            p_fault = ut.fault_point(avg_t_delay, short_circuit=False)

            p_values.append(p_fault)
            e_values.append(float(name))


In [4]:
#model performance
performance = ut.performance_metrics(e_values, p_values)
print('Model performance for all faults:', performance)

Model performance for all faults: Metrics(mse=2.5770553361071755, rmse=1.6053209448914494, mae=1.3416852885701298, irm=0.9975550122249389)
